In [1]:
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
import torch
import random
from torchvision import models, transforms
from numpy.linalg import norm
import numpy as np
import math
import time
from torch.utils.data import DataLoader, TensorDataset

In [2]:
import sys
root = '../../../'
sys.path.append(root)
from HelpfulFunctions.batchCreation import createBatch
from HelpfulFunctions.metrics import meanAveragePrecision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
X_train = np.load( root + "Features/train_features_vgg16_cifar10.npy" ) # Shape = (45000, 4096)
X_train_tensor = torch.tensor(X_train)
y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" ) # Shape = (45000,)


X_test = np.load( root + "Features/test_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
X_test_tensor = torch.tensor(X_test)
y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy" ) # Shape = (10000,)


In [4]:
def CreateDataset(root, num_classes, batch_size, train = True):
    if train == True:
        #Create X_train_tensor
        X_train = np.load( root + "Features/train_features_vgg16_cifar10.npy" ) # Shape = (45000, 4096)
        X_train_tensor = torch.tensor(X_train)

        #Create Y_train_tensor
        y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" ) # Shape = (45000,)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        y_train_tensor = torch.nn.functional.one_hot(y_train_tensor, num_classes) #One-Hot Encoded -> Shape = (45000, num_classes)

        #Create indices
        indices_train = torch.arange(len(X_train_tensor))

        dataset = TensorDataset(X_train_tensor, y_train_tensor, indices_train)
        train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return train_loader

    else:
        X_test = np.load( root + "Features/test_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
        X_test_tensor = torch.tensor(X_test)

        y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy" ) # Shape = (10000,)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)
        y_test_tensor = torch.nn.functional.one_hot(y_test_tensor, num_classes) #One-Hot Encoded -> Shape = (10000, num_classes)

        #Create indices
        indices_test = torch.arange(len(X_test_tensor))

        dataset = TensorDataset(X_test_tensor, y_test_tensor, indices_test)
        test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return test_loader

    #Missing implementation for Test and Validation
    

In [5]:
train_loader = CreateDataset(root, num_classes = 10, batch_size = 128)
test_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = False)


In [6]:
class CustomNN(nn.Module):
    def __init__(self):
        super(CustomNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(4096, 1024),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(1024, 48),    # Second fully connected layer to reduce to 4000
        )

    def forward(self, x):
        return self.fc_layers(x)

In [7]:
model = CustomNN().to(device)

In [8]:
class DSHLoss(torch.nn.Module):
    def __init__(self, train_size, n_classes, bit):
        super(DSHLoss, self).__init__()
        self.m = 2 * bit
        self.U = torch.zeros(train_size, bit).float().to(device)
        self.Y = torch.zeros(train_size, n_classes).float().to(device)

    def forward(self, u, y, ind, eta):
        self.U[ind, :] = u.data
        self.Y[ind, :] = y.float()

        dist = (u.unsqueeze(1) - self.U.unsqueeze(0)).pow(2).sum(dim=2)
        y = (y @ self.Y.t() == 0).float()

        loss = (1 - y) / 2 * dist + y / 2 * (self.m - dist).clamp(min=0)
        loss1 = loss.mean()
        loss2 = eta * (1 - u.abs()).abs().mean()

        return loss1 + loss2

In [9]:
def train_val(device, train_loader, train_size, batch_size, n_classes, bit, num_epoch, eta):


    optimizer = optim.RMSprop(model.parameters(), lr = 1e-5, weight_decay = 10 ** -5)

    criterion = DSHLoss(train_size, n_classes, bit)


    #model.train()

    #Best_mAP = 0

    for epoch in range(num_epoch):

        current_time = time.strftime('%H:%M:%S', time.localtime(time.time()))

        print("%s[%2d/%2d][%s] bit:%d, dataset:%s, training...." % (
            "DSH", epoch + 1, num_epoch, current_time, bit, "CIFAR"), end="")

        train_loss = 0
        for image, label, ind in train_loader:
            image = image.to(device)
            label = label.to(device)

            optimizer.zero_grad()
            u = model(image)


            loss = criterion(u, label.float(), ind, eta)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

        train_loss = train_loss / (train_size / batch_size)

        print("\b\b\b\b\b\b\b loss:%.5f" % (train_loss))

        #if (epoch + 1) % config["test_map"] == 0:
            #Best_mAP = validate(config, Best_mAP, test_loader, dataset_loader, net, bit, epoch, num_dataset)


In [10]:
trained_model = train_val(device, train_loader, train_size = 45000, batch_size = 128, n_classes = 10, bit = 48, num_epoch = 150, eta = 0.1)

DSH[ 1/150][10:25:16] bit:48, dataset:CIFAR, training... loss:4.33965
DSH[ 2/150][10:25:32] bit:48, dataset:CIFAR, training... loss:6.10590
DSH[ 3/150][10:25:48] bit:48, dataset:CIFAR, training... loss:5.43099
DSH[ 4/150][10:26:03] bit:48, dataset:CIFAR, training... loss:5.13345
DSH[ 5/150][10:26:17] bit:48, dataset:CIFAR, training... loss:4.93343
DSH[ 6/150][10:26:31] bit:48, dataset:CIFAR, training... loss:4.77908
DSH[ 7/150][10:26:46] bit:48, dataset:CIFAR, training... loss:4.65768
DSH[ 8/150][10:27:00] bit:48, dataset:CIFAR, training... loss:4.55718
DSH[ 9/150][10:27:14] bit:48, dataset:CIFAR, training... loss:4.47224
DSH[10/150][10:27:28] bit:48, dataset:CIFAR, training... loss:4.39560
DSH[11/150][10:27:42] bit:48, dataset:CIFAR, training... loss:4.32832
DSH[12/150][10:27:57] bit:48, dataset:CIFAR, training... loss:4.26781
DSH[13/150][10:28:12] bit:48, dataset:CIFAR, training... loss:4.21239
DSH[14/150][10:28:27] bit:48, dataset:CIFAR, training... loss:4.16178
DSH[15/150][10:28:41

In [11]:
trained_model = model.cpu()
hash_train = (trained_model(X_train_tensor)).sign()
hash_train = hash_train.detach().numpy()
hash_test = (trained_model(X_test_tensor)).sign()
hash_test = hash_test.detach().numpy()
y_train
y_test

array([3, 8, 8, ..., 5, 1, 7], dtype=int64)

In [12]:
map = meanAveragePrecision(training_hashes = hash_train, test_hashes = hash_test, test_labels = y_test, training_labels = y_train)

100%|██████████| 10000/10000 [02:00<00:00, 82.65it/s]


In [13]:
map

0.8482320072978633